# User Similarity Using Jaccard, MinHash & LSH

> *Data Mining*  
> *MSc in Data Science, Department of Informatics*  
> *Athens University of Economics and Business*

---

## *Table of Contents*

- [*1. Introduction*](#introduction)
    - [*1.1. Libraries*](#libraries)
    - [*1.2. Data*](#data)
    - [*1.3. Data Preprocessing*](#data_preprocessing)
- [*2. Compute Exact Jaccard Similarity*](#jaccard_similarity)
- [*3. Compute Similarity Using MinHash Signatures*](#minhash_similarity)
    - [*3.1. Using 50 Hash Functions*](#50_hash_functions)
    - [*3.2. Using 100 Hash Functions*](#100_hash_functions)
    - [*3.3. Using 200 Hash Functions*](#200_hash_functions)
    - [*3.4. Comments*](#comments_1)
- [*4. Locate Similar Users Using LSH Index*](#lsh_similarity)
    - [*4.1. LSH Instance 1*](#lsh_instance_1)
    - [*4.2. LSH Instance 2*](#lsh_instance_2)
    - [*4.3. Comments*](#comments_2)

---

## *Introduction* <a class='anchor' id='introduction'></a>

### *Libraries* <a class='anchor' id='libraries'></a>

In [1]:
import numpy as np
import pandas as pd

import time

from functions.data_preprocessing import *
from functions.jaccard_similarity import *
from functions.min_hash_similarity import *
from functions.lsh_similarity import *

### *Data* <a class='anchor' id='data'></a>

*The following two datasets will be used:*
- *`ratings.txt`*
- *`movies.txt`*

##### *Read ratings data*

In [2]:
# filepath
filepath = './data/ratings.txt'

# read the dataset
ratings = pd.read_csv(filepath, sep='\t', header=None, usecols=[0,1,2], names=['user_id','movie_id','rating'])

# shape
print(f'ratings.shape: {ratings.shape}')

# preview
ratings.head()

ratings.shape: (100000, 3)


,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


##### *Read movies data*

In [3]:
# filepath
filepath = './data/movies.txt'

# read the dataset
movies = pd.read_csv(filepath, sep='|', header=None, usecols=[0,1], names=['movie_id','title'], encoding='latin-1')

# shape
print(f'movies.shape: {movies.shape}')

# preview
movies.head()

movies.shape: (1682, 2)


,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


### *Data Preprocessing* <a class='anchor' id='data_preprocessing'></a>

- *In this step, we will preprocess the `ratings` dataframe*
- *In particular, we will transform its data into a `dictionary` format*
- *The keys of the dictionary will correspond to each user ID, while the values will be arrays containing all the movie IDs seen from each respective user*

##### *Preprocess `ratings`*

In [4]:
# execute function
user_movies = load_movies(ratings.iloc[:,:2])

# preview
len(user_movies)

943

---

## *Compute Exact Jaccard Similarity* <a class='anchor' id='jaccard_similarity'></a>

##### *Compute user similarity using Jaccard coefficient*

In [5]:
# execute function
jaccard_sim, jaccard_sim_threshold = user_similarity_using_jaccard_coefficient(user_movies)

##### *Display pair of users with a similarity score of at least 50%*

In [6]:
# pair of users: similarity
for k,v in jaccard_sim_threshold.items():
    print(f'{k}: {v}')

408_898: 0.8387096774193549
328_788: 0.6729559748427673
489_587: 0.6299212598425197
600_826: 0.5454545454545454
451_489: 0.5333333333333333
674_879: 0.5217391304347826
554_764: 0.5170068027210885
197_826: 0.512987012987013
197_600: 0.5
800_879: 0.5


##### *Get the movies seen from the most similar pair of users*

In [7]:
# execute function
get_the_movies_of_the_most_similar_pair_of_users(movies, jaccard_sim, user_movies)

Most similar pair of users: 408 - 898

Movies seen from the most similar pair of users:

 242: Kolya (1996)
 243: Jungle2Jungle (1997)
 258: Contact (1997)
 270: Gattaca (1997)
 271: Starship Troopers (1997)
 272: Good Will Hunting (1997)
 286: English Patient, The (1996)
 288: Scream (1996)
 294: Liar Liar (1997)
 300: Air Force One (1997)
 302: L.A. Confidential (1997)
 309: Deceiver (1997)
 310: Rainmaker, The (1997)
 312: Midnight in the Garden of Good and Evil (1997)
 313: Titanic (1997)
 315: Apt Pupil (1998)
 316: As Good As It Gets (1997)
 319: Everyone Says I Love You (1996)
 324: Lost Highway (1997)
 327: Cop Land (1997)
 328: Conspiracy Theory (1997)
 334: U Turn (1997)
 343: Alien: Resurrection (1997)
 347: Wag the Dog (1997)
 358: Spawn (1997)
 539: Mouse Hunt (1997)
 683: Rocket Man (1997)
 689: Jackal, The (1997)
 748: Saint, The (1997)
 751: Tomorrow Never Dies (1997)
1296: Indian Summer (1996)


---

## *Compute Similarity Using MinHash Signatures* <a class='anchor' id='minhash_similarity'></a>

### *Using 50 Hash Functions* <a class='anchor' id='50_hash_functions'></a>

##### *Compute user similarity using MinHash signatures*

In [8]:
# execute function
minhash_sim_50, minhash_sim_50_threshold = user_similarity_using_min_hash_signatures(user_movies,50)

##### *Display pair of users with a similarity score of at least 50%*

In [9]:
# pair of users: similarity
for k,v in minhash_sim_50_threshold.items():
    print(f'{k}: {v}')

328_788: 0.5151515151515151
408_898: 0.5151515151515151


##### *Report the number of False Positives (FP) and False Negatives (FN) (against the exact Jaccard similarity)*

In [10]:
# execute function
FP, FN = compute_the_number_of_false_positives_and_false_negatives(jaccard_sim, minhash_sim_50)

# print
print(f'There are {FP} False Positives (FP) and {FN} False Negatives (FN) (against the exact Jaccard similarity).')

There are 42218 False Positives (FP) and 386217 False Negatives (FN) (against the exact Jaccard similarity).


##### *Report the average number of False Positives (FP) and False Negatives (FN) for 5 different runs using different functions*

In [11]:
# start time
st = time.time()

# initialize empty lists
# to store FP and FN values
false_positives = []
false_negatives = []

for i in range(5):
    # compute user similarity using MinHash signatures
    minhash_sim_50, minhash_sim_50_threshold = user_similarity_using_min_hash_signatures(user_movies,50)
    # compute false positives and false negatives
    FP, FN = compute_the_number_of_false_positives_and_false_negatives(jaccard_sim, minhash_sim_50)
    # append the values from each iteration
    false_positives.append(FP)
    false_negatives.append(FN)

# calculate the averages
FP_avg = round(np.mean(false_positives))
FN_avg = round(np.mean(false_negatives))

# end time
et = time.time()

# print
print(f'The average number of False Positives is {FP_avg}, while the average number of False Negatives is {FN_avg}.')
print()
print(f'Elapsed time: {round(et-st)} secs.')

The average number of False Positives is 36228, while the average number of False Negatives is 392275.

Elapsed time: 59 secs.


### *Using 100 Hash Functions* <a class='anchor' id='100_hash_functions'></a>

##### *Compute user similarity using MinHash signatures*

In [12]:
# execute function
minhash_sim_100, minhash_sim_100_threshold = user_similarity_using_min_hash_signatures(user_movies,100)

##### *Display pair of users with a similarity score of at least 50%*

In [13]:
# pair of users: similarity
for k,v in minhash_sim_100_threshold.items():
    print(f'{k}: {v}')

408_898: 0.7543859649122807
328_788: 0.546875


##### *Report the number of False Positives (FP) and False Negatives (FN) (against the exact Jaccard similarity)*

In [14]:
# execute function
FP, FN = compute_the_number_of_false_positives_and_false_negatives(jaccard_sim, minhash_sim_100)

# print
print(f'There are {FP} False Positives (FP) and {FN} False Negatives (FN) (against the exact Jaccard similarity).')

There are 20110 False Positives (FP) and 408662 False Negatives (FN) (against the exact Jaccard similarity).


##### *Report the average number of False Positives (FP) and False Negatives (FN) for 5 different runs using different functions*

In [15]:
# start time
st = time.time()

# initialize empty lists
# to store FP and FN values
false_positives = []
false_negatives = []

for i in range(5):
    # compute user similarity using MinHash signatures
    minhash_sim_100, minhash_sim_100_threshold = user_similarity_using_min_hash_signatures(user_movies,100)
    # compute false positives and false negatives
    FP, FN = compute_the_number_of_false_positives_and_false_negatives(jaccard_sim, minhash_sim_100)
    # append the values from each iteration
    false_positives.append(FP)
    false_negatives.append(FN)

# calculate the averages
FP_avg = round(np.mean(false_positives))
FN_avg = round(np.mean(false_negatives))

# end time
et = time.time()

# print
print(f'The average number of False Positives is {FP_avg}, while the average number of False Negatives is {FN_avg}.')
print()
print(f'Elapsed time: {round(et-st)} secs.')

The average number of False Positives is 13898, while the average number of False Negatives is 415016.

Elapsed time: 112 secs.


### *Using 200 Hash Functions* <a class='anchor' id='200_hash_functions'></a>

##### *Compute user similarity using MinHash signatures*

In [16]:
# execute function
minhash_sim_200, minhash_sim_200_threshold = user_similarity_using_min_hash_signatures(user_movies,200)

##### *Display pair of users with a similarity score of at least 50%*

In [17]:
# pair of users: similarity
for k,v in minhash_sim_200_threshold.items():
    print(f'{k}: {v}')

408_898: 0.762114537444934
328_788: 0.5230769230769231


##### *Report the number of False Positives (FP) and False Negatives (FN) (against the exact Jaccard similarity)*

In [18]:
# execute function
FP, FN = compute_the_number_of_false_positives_and_false_negatives(jaccard_sim, minhash_sim_200)

# print
print(f'There are {FP} False Positives (FP) and {FN} False Negatives (FN) (against the exact Jaccard similarity).')

There are 7490 False Positives (FP) and 421558 False Negatives (FN) (against the exact Jaccard similarity).


##### *Report the average number of False Positives (FP) and False Negatives (FN) for 5 different runs using different functions*

In [19]:
# start time
st = time.time()

# initialize empty lists
# to store FP and FN values
false_positives = []
false_negatives = []

for i in range(5):
    # compute user similarity using MinHash signatures
    minhash_sim_200, minhash_sim_200_threshold = user_similarity_using_min_hash_signatures(user_movies,200)
    # compute false positives and false negatives
    FP, FN = compute_the_number_of_false_positives_and_false_negatives(jaccard_sim, minhash_sim_200)
    # append the values from each iteration
    false_positives.append(FP)
    false_negatives.append(FN)

# calculate the averages
FP_avg = round(np.mean(false_positives))
FN_avg = round(np.mean(false_negatives))

# end time
et = time.time()

# print
print(f'The average number of False Positives is {FP_avg}, while the average number of False Negatives is {FN_avg}.')
print()
print(f'Elapsed time: {round(et-st)} secs.')

The average number of False Positives is 5173, while the average number of False Negatives is 423885.

Elapsed time: 208 secs.


### *Comment*  <a class='anchor' id='comment_1'></a>

- *Let's observe the average results obtained from using 50, 100 and 200 hash functions*
- *There are two (2) observations here:*
    - *1. The more hash functions we use, the smaller the number of False Positives (36228 > 13898 > 5173)*
    - *2. The more hash functions we use, the higher the number of False Negatives (392275 < 415016 < 423885)*
- *By adding more hash functions, we "build" additional layers of checking, and hence MinHash becomes a better filtering mechanism*
- *On the other hand, this increases the false negative rate, because the requirement for a full signature match is becoming more stringent and less likely*

---

## *Locate Similar Users Using LSH Index* <a class='anchor' id='lsh_similarity'></a>

### *LSH Instance 1* <a class='anchor' id='lsh_instance_1'></a>

- $b = 25$
- $r = 8$

##### *Locate similar users using LSH index*

In [20]:
# initialize variables
b, r = 25, 8

# execute function
lsh_sim_258_threshold, true_pairs, similarity_evaluations = user_similarity_using_lsh(user_movies, b, r)

##### *Display pairs of users with a similarity score of at least 50%*

In [21]:
# pair of users: similarity
for k,v in lsh_sim_258_threshold.items():
    print(f'{k}: {v}')

408_898: 0.8387096774193549
489_587: 0.6299212598425197
554_764: 0.5170068027210885


##### *Report the average number of true pairs (TP) and similarity evaluations for 5 different runs using different functions*

In [22]:
# start time
st = time.time()

# initialize empty lists to store
# the number of true pairs
# and the number of similarity evaluations
true_positives = []
sim_evaluations = []

for i in range(5):
    # locate similar users using LSH index
    _, true_pairs, similarity_evaluations = user_similarity_using_lsh(user_movies, b, r)
    # append the values from each iteration
    true_positives.append(true_pairs)
    sim_evaluations.append(similarity_evaluations)

# calculate the averages
TP_avg = round(np.mean(true_positives))
SE_avg = round(np.mean(sim_evaluations))

# end time
et = time.time()

# print
print(f'The average number of true pairs is {TP_avg}, while the average number of similarity evaluations is {SE_avg}.')
print()
print(f'Elapsed time: {round(et-st)} secs.')

The average number of true pairs is 3, while the average number of similarity evaluations is 40.

Elapsed time: 169 secs.


### *LSH Instance 2*  <a class='anchor' id='lsh_instance_2'></a>

- $b = 40$
- $r = 5$

##### *Locate similar users using LSH index*

In [23]:
# initialize variables
b, r = 40, 5

# execute function
lsh_sim_405_threshold, true_pairs, similarity_evaluations = user_similarity_using_lsh(user_movies, b, r)

##### *Display pairs of users with a similarity score of at least 50%*

In [24]:
# pair of users: similarity
for k,v in lsh_sim_405_threshold.items():
    print(f'{k}: {v}')

408_898: 0.8387096774193549
328_788: 0.6729559748427673
489_587: 0.6299212598425197
600_826: 0.5454545454545454
451_489: 0.5333333333333333
674_879: 0.5217391304347826
554_764: 0.5170068027210885
197_826: 0.512987012987013
197_600: 0.5


##### *Report the average number of true pairs (TP) and similarity evaluations for 5 different runs using different functions*

In [25]:
# start time
st = time.time()

# initialize empty lists to store
# the number of true pairs
# and the number of similarity evaluations
true_positives = []
sim_evaluations = []

for i in range(5):
    # locate similar users using LSH index
    _, true_pairs, similarity_evaluations = user_similarity_using_lsh(user_movies, b, r)
    # append the values from each iteration
    true_positives.append(true_pairs)
    sim_evaluations.append(similarity_evaluations)

# calculate the averages
TP_avg = round(np.mean(true_positives))
SE_avg = round(np.mean(sim_evaluations))

# end time
et = time.time()

# print
print(f'The average number of true pairs is {TP_avg}, while the average number of similarity evaluations is {SE_avg}.')
print()
print(f'Elapsed time: {round(et-st)} secs.')

The average number of true pairs is 7, while the average number of similarity evaluations is 2192.

Elapsed time: 225 secs.


### *Comment*  <a class='anchor' id='comment_2'></a>

- *The space required by minhashes is $O(b*r)$*
- *Increasing $b$ gives users mini-signatures more chances to match, so it let’s in candidate pairs with lower similarity scores*
- *Increasing $r$ makes the match criteria stricter, restricting to higher similarity scores*
- *LSH helps reduce the complexity of comparing all pairs in datasets containing millions or billions of samples to calculate their similarity*
- *The solution it provides is an* ***approximate search***
- *Rather than comparing every vector in the sample* ***(exhaustive search),*** *it approximates and limits the search scope to only the most relevant vectors*
- *Therefore, by using LSH in enormous datasets, we gain computational speed at the cost of some accuracy*

---

*Thank you!*

---